# OnlineStats.jl

_git:_ https://github.com/joshday/OnlineStats.jl                          
_documentation:_ http://joshday.github.io/OnlineStats.jl/stable/index.html

## Summary

Online Stats provide multipel analysis algorithms together with a framework to apply them to online data, this means that instead of requiring all the data to be loaded and analysed at once, one can feed it over time and update the current stat. 
(Note: we follow the documnetation's convention which uses _stat_ to name any statistics/models that the library can handle)

This is an extremely interesting feature since it not only allows to work with data coming through a live feed and constantly updating the _stat_, it also allows the user to divide the input up into smaller parts if it is too large to be handled at once, for instance for memory usage. Additionally, the library allows merging of its _stats_, making parallelization extremely simple, since the data can simply be divided into subsets, each fitted separately, and then merged at the end.


The library includes stats ranging from mean and variance, to statistical learning methods such as SVM and multivariate analysis such as PCA. A full list of models it supports can be found [here](http://joshday.github.io/OnlineStats.jl/stable/stats_and_models.html#Statistics-and-Models-1)


## Usage

### Basic
The package works mostly using only two functions

```julia
    s = Series( Stats1(), Stats2(), .. (, x, Weight() ) )
```

Here we instantiate a Series, using $x$ as initial input (optional), any number of _models_ from the models list listed above, and a _weight_ function, which determines how the weight of observation varies as more observations are added.


Next we are fitting the new inputs y to the already instantiated series s. Can additionally tell the fitting to use columns or rows for observations (defaults to row).

```julia
    fit!(s, y (,Cols()/Rows()) )
```

Finally, we can get the value of a stat at any point by invoking the _value_ function:

```julia
    value(s)
```

### Merging

If two series follow the same models, they can be merged. Additionally, options can be given as two the merging method

```julia  
    # Default
    merge!(s1, s2, :append)  
    # equivalent to fit!(s1, y1); fit(s1, y2);

    
    # Using weighted average
    merge!(s1, s2, :weighted)


    # Treat s2 as a singleton
    merge!(s1, s2, :singleton)


    # Provide ratio of influence s2 should have
    merge!(s1, s2, .5)
    
```

### Weights

The choice of weights is quite important in an online method since every merge and update will use it to attribute the amount of influence of the observation. OnlineStats already contains a large library of weights, and additionally allows for custom weight functions to be passed. More information can be found [here](http://joshday.github.io/OnlineStats.jl/stable/weights.html)

Weights are passed as arugment to the _Series_ function, for example using mean and variance as _stats_:

```julia
    Series(x, WeightFunction(), Mean(), Variance()) 
```

The following plot shows the different weight functions and how they change with the number of data points added.

<img src="https://user-images.githubusercontent.com/8075494/29486708-a52b9de6-84ba-11e7-86c5-debfc5a80cca.png" width="600"/>


### Parallelization

Parallelization is then simply done by fitting the series made of the same _stats_ but given different subsets of the dataset. Note that not all _stats_ will give the exact result when merged compared to if they were fitted as one. The models that do allow for exact merging are of the subtype ExactStat.

```julia
    y1 = randn(10_000)
    y2 = randn(10_000)
    y3 = randn(10_000)

    s1 = Series(Mean(), Variance(), Hist(50))
    s2 = Series(Mean(), Variance(), Hist(50))
    s3 = Series(Mean(), Variance(), Hist(50))

    fit!(s1, y1)
    fit!(s2, y2)
    fit!(s3, y3)

    merge!(s1, s2)  # merge information from s2 into s1
    merge!(s1, s3)  # merge information from s3 into s1

```

### Statistical Learning Methods

The objective function that OnlineStats tries to minimize is 

$$ \frac{1}{n} \sum_{i}^{n} f_i(\beta) + \sum_{j=1}^{p} \lambda_{p} g(\beta_j) $$

Choosing the right penalties and losses can then produce a multitude of models, including Ridge, Lasso, SVMs, Logistic.

```julia
    StatLearn(p::Int, args)
    args:
        loss = .5 * L2DistLoss() # any Loss from LossFunctions.jl
        penalty = L2Penalty() # any Penalty (which has a prox method) from PenaltyFunctions.jl.
        λ = fill(.1, p) # a Vector of element-wise regularization parameters
        updater = SGD() # SGD, ADAGRAD, ADAM, ADAMAX, MSPI

    ### Usage ### 

    o = StatLearn(10, .5 * L2DistLoss(), L1Penalty(), fill(.1, 10), SGD())
    s = Series(o)
    fit!(s, x, y)
    coef(o)
    predict(o, x)
```

### Other methods


Linear regression [_(documentation)_](http://joshday.github.io/OnlineStats.jl/stable/api.html#OnlineStats.LinRegBuilder)
```julia 
    lr = LinRegBuilder(p) # p = number of dimensions
    Series((x,y), lr) # Will do simple OLS
    value(lr)

    # Can provide additional arguments with coef function
    Series(x,lr)
    coef(lr, λ, y=[7], x=[1,2,3]) # Will use columns 1,2,3 to predict column 7 of x
    coef(lr, .5) # Make Ridge by specifying λ (can be float of vector of p dimensions)
    value(lr)
```

Calculate covariance matrix and PCA
```julia
    y = randn(1000, 5)
    o = CovMatrix(5)
    Series(y, o)

    # PCA & tranformation
    d_out = 2
    corr = cor(o)
    evals, evecs = eig(corr)
    pc = sortperm(evals, rev=true)[1:d_out]
    T = y * evecs[:,pc];
```


